Importing Libraries

In [ ]:
import math
import actionlib
import actionlib.msg
import assignment_2_2022.msg
import rospy
import time
import sys
import select
from geometry_msgs.msg import Point, Pose, Twist
from sensor_msgs.msg import LaserScan
from nav_msgs.msg import Odometry
from tf import transformations
from std_srvs.srv import *
from assignment_2_2022.msg import Robot_pos_vel

import tf
from tf.transformations import quaternion_matrix
import numpy as np
from matplotlib.animation import FuncAnimation
import numpy as np
import matplotlib as mpl

import jupyros as jr
import rospy
import ipywidgets as widgets
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
from matplotlib import pyplot as plt


Defining variables - starting node and publisher

In [ ]:
global goal
global client
global publisher
global msg
global cancelled_goals
global reached_goals
global goal_list

x_plot = []
y_plot = []
xg_plot = []
yg_plot = []

# Initializes a rospy node so that the SimpleActionClient can publish and subscribe over ROS.
rospy.init_node('action_client_py')

# Publisher to /pos_and_vel topic the position and velocity
pub = rospy.Publisher("/robot_pos_vel", Robot_pos_vel, queue_size=10)

Callback Function

In [ ]:
def callback(msg):
    global pub
    global x_plot, y_plot
    
    # Fill the matrix plot with the coordinates
    x_plot.append(msg.pose.pose.position.x)
    y_plot.append(msg.pose.pose.position.y)
    
    # Get the position 
    pos = msg.pose.pose.position
    
    # Get the linear velocity
    linear_velocity = msg.twist.twist.linear
    
    # Create custom message
    robot_pos_vel = Robot_pos_vel()
    robot_pos_vel.pos_x = pos.x
    robot_pos_vel.pos_y = pos.y
    robot_pos_vel.vel_x = linear_velocity.x
    robot_pos_vel.vel_y = linear_velocity.y
    
    # Publish the custom message
    pub.publish(robot_pos_vel)


Subscriber to the topic /odom

In [ ]:
# Subscriber to /odom topic to get position and velocity
jr.subscribe("/odom", Odometry, callback)

Setting action client + interface to set/cancel goal

In [ ]:
# Creates the SimpleActionClient, passing the type of the action to the constructor.
client = actionlib.SimpleActionClient('/reaching_goal', assignment_2_2022.msg.PlanningAction)

# Waits until the action server has started up and started listening for goals.
client.wait_for_server()

goal = assignment_2_2022.msg.PlanningGoal()

# Goal coordinates
x = widgets.FloatText(description = "x:")
y = widgets.FloatText(description = "y:")
display(x, y)

# Widgets Buttons
send_button = Button(description='Send goal',tooltip='Send goal',layout=Layout(width='50%', height='100px', grid_area='b1'),button_style='info')
cancel_button = Button(description='Cancel goal',tooltip='Cancel goal',layout=Layout(width='50%',height='100px', grid_area='b2'),button_style='danger')
display(send_button, cancel_button)

# if send button is clicked 
def on_button_send_clicked(b):
    
    goal.target_pose.pose.position.x = x.value
    goal.target_pose.pose.position.y = y.value
    
    #update goal list
    goal_list.value = goal_list.value + "Goal x: " + str(x.value) + "  y: " + str(y.value) + "\n"
    
    #send the goal returned by set_goal()
    client.send_goal(goal)
    
    # add goal to coordinates to plot
    xg_plot.append(x.value)
    yg_plot.append(y.value)
    
    x.disabled = True
    y.disabled = True
    send_button.disabled = True
    cancel_button.disabled = False
    

send_button.on_click(on_button_send_clicked)


def on_cancel_button_clicked(b):
    client.cancel_goal()
    
cancel_button.on_click(on_cancel_button_clicked)

Update values + check goal 

In [ ]:
def update_num(reached_goals, cancelled_goals):
    
    goal_list.value = goal_list.value +  str(cancelled_goals)+ "  Goal cancelled!\n"
    goal_list.value = goal_list.value +  str(reached_goals)+ "  Goal reached!\n"

In [ ]:
def checkGoalResult(msg):
    global cancelled_goals, reached_goals
    
    # Get the status 
    status = msg.status.status

    # If status is 2 the goal is canceled
    if status == 2:
        
        cancelled_goals = cancelled_goals + 1
        
        x.disabled = False
        y.disabled = False
        
        send_button.disabled = False
        cancel_button.disabled = True
        
        

    # If status is 3 the goal is reached
    elif status == 3:
        
        reached_goals = reached_goals + 1
        
        x.disabled = False
        y.disabled = False
        
        send_button.disabled = False
        cancel_button.disabled = True
        
        

    update_num(reached_goals, cancelled_goals)
    
reached_goals, cancelled_goals = 0,0

Subscriber for result

In [ ]:
jr.subscribe("/reaching_goal/result", assignment_2_2022.msg.PlanningActionResult, checkGoalResult)


Laser Function

In [ ]:
dmin = widgets.FloatText(description = "Distance:", disabled = True)
ang = widgets.FloatText(description = "Angle:", disabled = True)

def laserCallback(scan):
    min_range = 100
    angle = 100
    for at, x in enumerate(scan.ranges):
        if x < min_range and x > scan.range_min:
            min_range = x
            angle = scan.angle_min + scan.angle_increment * at
    
    dmin.value = min_range
    ang.value = angle

Setting subscriber to the topic /odom

In [ ]:
jr.subscribe('/scan', LaserScan, laserCallback)

Display real-time error 

In [ ]:
widgets.HBox([dmin, ang], description = "Distance from obstacle")


Display real-time robot Position and Velocity

In [ ]:
posx = widgets.FloatText(description = "Pos x:", disabled = True)
posy = widgets.FloatText(description = "Pos y:", disabled = True)
velx = widgets.FloatText(description = "Vel x:", disabled = True)
vely = widgets.FloatText(description = "Vel y:", disabled = True)
    
widgets.HBox([widgets.VBox([posx, posy]), widgets.VBox([velx, vely])])

 Update and display a plot based on incoming data

In [ ]:
class Visualiser:
    
    def __init__(self):
        self.x_data, self.y_data = [] , []
    
    def vis_callback(self, data):
        self.y_data.append(data.pos_y)
        self.x_data.append(data.pos_x)
        posx.value = data.pos_x
        posy.value = data.pos_y
        velx.value = data.vel_x
        vely.value = data.vel_y
        
    def update_plot(self, frame):
        self.ln.set_data(self.x_data, self.y_data)
        return self.ln

In [ ]:
vis = Visualiser()
sub = jr.subscribe('/robot_pos_vel', Robot_pos_vel, vis.vis_callback)

Plot real-time position of robot + actual goal

In [ ]:
# if the plot does not update, just re - run this cell

np_x_plot = np.array(x_plot) 
np_y_plot = np.array(y_plot)

fig = plt.figure()

ax = fig.add_axes([0,0,1,1])
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)

ax.plot(np_x_plot,np_y_plot,label='trajectory')
ax.scatter(xg_plot,yg_plot, color='red',label='goal')

ax.set_title("my_robot")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.legend()

Plot of cancelled/reached goals with plt.bar

In [ ]:
# if the plot does not update, just re - run this cell

n_goal = [reached_goals, cancelled_goals]
goals = ['reached goal', 'cancelled goals']

plt.bar(goals, n_goal, color=['green', 'red'])

plt.xlabel('goals type')
plt.ylabel('number')
plt.title('Number goals reached/cancelled')

plt.show()

Displaying cancelled/reached goals

In [ ]:
goal_list = widgets.Textarea(value = "Goal list:\n", disabled = True, style = dict(text_color = 'red') )

display(goal_list)